# Retrieving AOD based on SARA Algorithm - Final Run


AOD could be derived from the following function. 
$
\tau_{\alpha} = \frac{4\mu_s\mu_v}{\omega_0P_\alpha}[\rho_{TOA}-\rho_{Ray}-\frac{e^{-(\tau_R+\tau_\alpha)/\mu_s}e^{(\tau_R+\tau_\alpha)/\mu_v}\rho_s}{1-\rho_s(0.92\tau_R+(1-g)\tau_\alpha)e^{-(\tau_R+\tau_\alpha)}}]
$

Now that we have $\omega$ and $g$ calculated, the only unknown variable in this function is the AOD value, which is what we are looking for.

## 0. Set up environment

#### 0.1 Importing necessary python libraries and functions

In [2]:
import numpy as np
import readData2 as rd2
import matplotlib.pyplot as plt
import math

#### 0.2 Restore the parameter omega and g from numpy file

In [3]:
omega = np.load("para_omega.npy")
g = np.load("para_g.npy")

# Rayleigh Optical Depth
tau_R = 0.09847234720729856

## 1. AOD Retrieval

In [9]:
def retrieve_aod(dofy):
    mu_s = rd2.load(rd2.read_mu_s(dofy))
    mu_v = rd2.load(rd2.read_mu_v(dofy))
    P_a = rd2.load(rd2.read_P_a(dofy))
    toa = rd2.load(rd2.read_myd02(dofy))
    ray = rd2.load(rd2.read_ray(dofy))
    sur = rd2.load(rd2.read_myd09(dofy))
    g_ = g[dofy - 298]
    omega_ = omega[dofy - 298]
    rows, cols = np.shape(toa)
    
    
    def calculate_right(aod, i, j):
        numerator = math.pow(math.e, -(tau_R + aod)/mu_s[i, j]) * math.pow(math.e, -(tau_R + aod)/mu_v[i, j]) * sur[i, j]
        denominator = 1 - sur[i, j] * (0.92 * tau_R + (1 - g_) * aod) * math.pow(math.e, -(tau_R + aod))
        right = 2 * toa[i, j] - ray[i, j] - 0.2 * numerator / denominator
        right = 4 * mu_s[i, j] * mu_v[i, j] / (omega_ * P_a[i, j]) * right
        return right
        
    
    aod_mat = np.zeros((rows, cols))
    aod_num_list = np.linspace(0.0, 0.3, 61)
    for i in range(0, rows):
        if (i%100 == 0):
            print("Now we are working on the row {} of the day {}...".format(i, dofy))
        for j in range(0, cols):
            if sur[i, j] <= 0:
                aod_mat[i, j] = -100
                continue
            min_error = 10000000
            best_aod = -100
            for aod in aod_num_list:
                right = calculate_right(aod, i, j)
                error = math.fabs(right - aod)
                if error < min_error:
                    min_error = error
                    best_aod = aod
            aod_mat[i, j] = best_aod
    
    rd2.matrix_to_geo_tiff("AOD/AOD_2019_{:03d}.tif".format(dofy), aod_mat)
    return aod_mat


In [10]:
def retrieve_batch():
    aod_list = []
    for i in range(298, 310):
        print()
        print("**** Start working on the day {}... ****".format(i))
        print()
        aod_list.append(retrieve_aod(i))
    return aod_list

In [ ]:
aod_list = retrieve_batch()


**** Start working on the day 298... ****

Now we are working on the row 0 of the day 298...
Now we are working on the row 100 of the day 298...
Now we are working on the row 200 of the day 298...
Now we are working on the row 300 of the day 298...
Now we are working on the row 400 of the day 298...
Now we are working on the row 500 of the day 298...
Now we are working on the row 600 of the day 298...

**** Start working on the day 299... ****

Now we are working on the row 0 of the day 299...
Now we are working on the row 100 of the day 299...
Now we are working on the row 200 of the day 299...
Now we are working on the row 300 of the day 299...
Now we are working on the row 400 of the day 299...


In [ ]:
fig, ax = plt.subplots(4,3)
for 